In [24]:
import pandas as pd
import os
import json
from sklearn.model_selection import train_test_split

In [2]:
save_reports = './data/reports_saved.csv'

df = pd.read_csv(save_reports)
df = df[['name', 'definition']]
df

,name,definition
0,Abbrev by day,"{""performance_summary"":{""cols"":[{""id"":""fact_im..."
1,Full Performance Summary,"{""performance_summary"":{""cols"":[{""id"":""fact_im..."
2,Abbrev by merchant,"{""performance_summary"":{""cols"":[{""id"":""fact_im..."
3,FPS By Day,"{""performance_summary"":{""cols"":[{""id"":""fact_im..."
4,FPS by Affiliate Default Website,"{""performance_summary"":{""cols"":[{""id"":""fact_im..."
...,...,...
1430,test report for phpunit,"{""performance_summary"":{""cols"":[{""id"":""fact_im..."
1431,test report for phpunit,"{""performance_summary"":{""cols"":[{""id"":""fact_im..."
1432,test report for phpunit,"{""performance_summary"":{""cols"":[{""id"":""fact_im..."
1433,test report for phpunit,"{""performance_summary"":{""cols"":[{""id"":""fact_im..."


In [3]:
def remove_duplicate_definitions(df):
    # Convert the nested JSON objects to strings
    df['definition_str'] = df['definition'].apply(lambda x: json.dumps(x, sort_keys=True))

    # Identify unique and duplicated strings
    unique_definitions = df['definition_str'].duplicated(keep=False)

    # Filter the DataFrame to keep only rows with unique 'definition's
    df_unique = df[~unique_definitions]

    # Drop the 'definition_str' column as it's no longer needed
    df_unique = df_unique.drop(columns=['definition_str'])

    return df_unique

# Call the function
df_unique = remove_duplicate_definitions(df)

In [4]:
df_unique

In [6]:
df_unique.rename(columns={'definition': 'code'}, inplace=True)

In [29]:
system_prompt = """
Title: Human Readable Query From JSON

Objective: Generate a human-readable query from the provided JSON object which represents a data request for a performance summary report by affiliate.

JSON Structure Overview:
- report_name: Specifies the main objective of the query.
- filters: Contains date or time-related information and specific entities being queried.
- cols: Contains metrics and dimensions to be included in the report.
- format, limit, offset: Specifies how the data should be formatted and how many records are to be displayed.

Instructions:

1. Identify the Main Objective:
   - Extract the value of the report_name key to understand the main objective of the query.

2. Identify Date or Time Frame:
   - Look for the filters key and extract date or time-related information. Check for keys like dim_date-mm_dd_yyyy, op, and period to understand the time frame being queried.

3. Identify Specific Entities:
   - Within the filters key, look for specific entities being queried, like dim_merchant-merchant_uuid and extract the value to identify the specific merchant being queried.

4. Identify Metrics and Dimensions:
   - Look for the cols key and extract the metrics and dimensions to be included in the report. Metrics and dimensions usually have keys like name and alias for human-readable references.

5. Identify Format and Limits:
   - Extract the values of keys like format, limit, and offset to understand how the data should be formatted and how many records are to be displayed.

6. Compose the Query:
   - Start with the main objective, mention the date or time frame, specify the entities, list down the metrics and dimensions, and mention the format and limits.

7. Review and Refine:
   - Review the composed query to ensure it captures all the necessary information from the JSON.
   - Refine the wording to ensure clarity and readability.

8. Do Not Inlude Anything besides the Query in your response.

9. Do Not Specify the data must be returned in JSON format, it is implied.
"""

In [16]:
df_unique = df_unique.sort_values(by='name')

In [25]:
df1, df2 = train_test_split(df_unique, test_size=0.5, random_state=42, shuffle=True)

In [68]:
import time
import openai
from tqdm import tqdm
import pandas as pd

class TokenManager:
    def __init__(self, maximum_tokens_per_minute=10000):
        self.maximum_tokens_per_minute = maximum_tokens_per_minute
        self.tokens_utilized_in_the_last_minute = 0
        self.timestamp_of_the_last_request = time.time()
        self.total_tokens_used = 0  # Initialize total tokens used to 0

    def _reset_tokens_if_needed(self):
        current_timestamp = time.time()
        time_elapsed_since_last_request = current_timestamp - self.timestamp_of_the_last_request
        
        if time_elapsed_since_last_request >= 60:
            self.tokens_utilized_in_the_last_minute = 0
            self.timestamp_of_the_last_request = current_timestamp

    def calculate_tokens_available(self):
        self._reset_tokens_if_needed()
        tokens_available = self.maximum_tokens_per_minute - self.tokens_utilized_in_the_last_minute
        return tokens_available

    def register_tokens_used(self, tokens_used):
        if not isinstance(tokens_used, int) or tokens_used < 0:
            raise ValueError("Invalid token count")
        
        self._reset_tokens_if_needed()
        self.tokens_utilized_in_the_last_minute += tokens_used  # Update the tokens utilized in the last minute
        self.total_tokens_used += tokens_used  # Update the total tokens used



In [69]:

class OpenAIInteraction:
    def __init__(self, token_manager, output_csv, model="gpt-4"):
        self.token_manager = token_manager
        self.model = model
        self.output_csv = output_csv

    def request_response(self, code_snippet, system_prompt, docstring=None):
        while self.token_manager.calculate_tokens_available() < 2500:  # Assume a minimum of 2500 tokens per request
            time.sleep(1)
            print('Waiting for tokens to be available...')

        full_prompt = f"{system_prompt}"

        messages = [
            {"role": "system", "content": full_prompt},
            {"role": "user", "content": code_snippet}
        ]

        try:
            response = openai.ChatCompletion.create(
                model=self.model,  
                messages=messages
            )
            tokens_used = response['usage']['total_tokens']
            self.token_manager.register_tokens_used(tokens_used)
            print(f'Response received, {tokens_used} tokens used, {self.token_manager.total_tokens_used} tokens used in total this run.')
            return response
        except Exception as e:
            print(f"An error occurred: {str(e)}")

    def process_code_snippets(self, dataframe, target_col, system_prompt):
        if target_col not in dataframe.columns:
            print(f"Error: {target_col} does not exist in the dataframe")
            return

        # Check if the output CSV file exists, if not, create it
        if not os.path.exists(self.output_csv):
            with open(self.output_csv, 'w') as f:
                f.write(','.join(dataframe.columns.tolist() + ['output']) + '\n')  # Write headers

        total_snippets = len(dataframe)
        print(f'Processing {total_snippets} code snippets...')

        for _, row in tqdm(dataframe.iterrows(), total=total_snippets):  # Using tqdm for progress bar
            # Read the output CSV to check if the row has been processed
            output_df = pd.read_csv(self.output_csv)
            if any(output_df[target_col] == row[target_col]):
                print(f'Skipping row {_}, already processed.')
                continue  # Skip this row if it has been processed

            response = self.request_response(row[target_col], system_prompt)
            if response:
                # Append the new data to the output CSV
                row_output = pd.DataFrame([row.tolist() + [response]], columns=dataframe.columns.tolist() + ['output'])
                row_output.to_csv(self.output_csv, mode='a', header=False, index=False)
            else:
                print(f'Failed to process row {_}.')

    def aggregate_row_data(self, row, columns=None):
        if columns is None:
            columns = row.index.tolist()
        return '\n'.join([f"{col}: {row[col]}" for col in columns])

    def row_already_processed(self, row, unique_cols):
        if not os.path.exists(self.output_csv):
            return False
        output_df = pd.read_csv(self.output_csv)
        for _, output_row in output_df.iterrows():
            if all(str(row[col]) == str(output_row[col]) for col in unique_cols):
                return True
        return False

    def process_database_rows(self, dataframe, system_prompt,columns=None, unique_cols=None):
        if not os.path.exists(self.output_csv):
            with open(self.output_csv, 'w') as f:
                f.write(','.join(dataframe.columns.tolist() + ['output']) + '\n')  # Write headers

        if unique_cols is None:
            unique_cols = dataframe.columns.tolist()

        total_rows = len(dataframe)
        print(f'Processing {total_rows} database rows...')

        for _, row in tqdm(dataframe.iterrows(), total=total_rows):
            if self.row_already_processed(row, unique_cols):
                print(f'Skipping row {_}, already processed.')
                continue

            # Aggregate row data into a single string document
            user_prompt = self.aggregate_row_data(row, columns)

            response = self.request_response(user_prompt, system_prompt)
            if response:
                output_content = response.get('choices', [{}])[0].get('message', {}).get('content', '')
                # Append the new data to the output CSV
                row_output = pd.DataFrame([row.tolist() + [output_content]], columns=dataframe.columns.tolist() + ['output'])
                row_output.to_csv(self.output_csv, mode='a', header=False, index=False)
            else:
                print(f'Failed to process row {_}.')

        print('Processing completed.')

In [40]:
# Usage
# token_manager = TokenManager()
# openai_interaction = OpenAIInteraction(token_manager)
# openai_interaction.process_code_snippets(df1, system_prompt)

## Lets Grab the prepared columns and create a RAG retrieval system

In [107]:
prepared_columns = pd.read_csv('./data/reports_prepared_columns.csv')
new_token_manager = TokenManager()
new_openai_interaction = OpenAIInteraction(new_token_manager, './data/prepared_columns_processed_v200.csv')
prepared_columns_system_prompt = '''
Please provide a very short description (15 words or less) for the following rows based on the provided data.
Dont mention the environment, visible to, and accesible to. 
- If the "display_groups"={metrics} (and NOTHING ElSE), you should mention Avantmetrics in the Description returned
- If "display_groups"={performance} (and NOTHING ELSE), you should mention Performance in the Description returned.
- Otherwise you dont need to mention the display group in the description returned.
'''
columns_to_aggregate = ['name', 'definition', 'description', 'display_groups', 'table_id', 'classifier']
new_openai_interaction.process_database_rows(prepared_columns, prepared_columns_system_prompt, columns=columns_to_aggregate)

Processing 165 database rows...


  1%|          | 1/165 [00:02<06:07,  2.24s/it]

Response received, 199 tokens used, 199 tokens used in total this run.


  1%|          | 2/165 [00:04<05:51,  2.15s/it]

Response received, 179 tokens used, 378 tokens used in total this run.


...

Response received, 333 tokens used, 40030 tokens used in total this run.


 99%|█████████▉| 164/165 [05:35<00:02,  2.12s/it]

Response received, 192 tokens used, 40222 tokens used in total this run.


100%|██████████| 165/165 [05:37<00:00,  2.05s/it]

Response received, 184 tokens used, 40406 tokens used in total this run.
Processing completed.


In [113]:
described_prepared_columns = pd.read_csv('./data/prepared_columns_processed_v200.csv')

def safe_json_loads(s):
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        return None
    
def extract_content(output):
    # Try to load the string as JSON
    try:
        output_dict = json.loads(output)
        # Try to extract the "content" field
        if 'choices' in output_dict and output_dict['choices']:
            return output_dict['choices'][0]['message']['content']
    except (json.JSONDecodeError, TypeError, KeyError):
        # If it's not a JSON string or if the extraction fails, return the original string
        return output
    
described_prepared_columns = described_prepared_columns[described_prepared_columns['output'].str.strip() != ""]
described_prepared_columns['output'].apply(safe_json_loads)
# Apply the function to each row in the 'output' column
described_prepared_columns['extracted_content'] = described_prepared_columns['output'].apply(extract_content)
described_prepared_columns['extracted_content'] = described_prepared_columns.apply(lambda row: row['output'] if pd.isnull(row['extracted_content']) or row['extracted_content'] == "" else row['extracted_content'], axis=1)
described_prepared_columns['extracted_content'] = described_prepared_columns['extracted_content'].apply(lambda x: x[1:-1] if isinstance(x, str) and x.startswith('"') and x.endswith('"') else x)

described_prepared_columns = described_prepared_columns.drop(columns=['output'])
described_prepared_columns['definition'] = described_prepared_columns['extracted_content']
described_prepared_columns = described_prepared_columns.drop(columns=['extracted_content'])
described_prepared_columns.columns.values

# described_prepared_columns.to_csv('./data/prepared_columns_described_v200.csv', index=False)

In [36]:
# import pandas as pd
# import matplotlib.pyplot as plt
# from matplotlib.backends.backend_pdf import PdfPages
# import textwrap

# class StretchyPDF:
#     """
#     A class for generating a PDF document from a Pandas DataFrame. Each row of the DataFrame is converted to text and added as a page in the PDF document.
    
#     Attributes:
#         pdf_pages (PdfPages): A PdfPages object from matplotlib.backends.backend_pdf, used to save figures to a PDF file.
    
#     Methods:
#         estimate_fig_dimensions(text, char_width=0.1, line_height=0.2):
#             Estimate the dimensions of the figure based on the text content.
            
#         add_text_to_pdf(doc_text):
#             Create a matplotlib figure, add text to it, and save the figure to the PDF file.
            
#         create_document(row):
#             Convert a row from the DataFrame to a formatted text document.
            
#         create_pdf(data):
#             Iterate through rows of a Pandas DataFrame, convert each row to text, and add it to the PDF document.
            
#         close_pdf():
#             Close the PDF file after all pages have been added.

#     Example:
#         >>> df = pd.DataFrame({'Name': ['Alice', 'Bob'], 'Age': [30, 25], 'Occupation': ['Engineer', 'Doctor']})
#         >>> stretchy_pdf = StretchyPDF('output.pdf')
#         >>> stretchy_pdf.create_pdf(df)
#         >>> stretchy_pdf.close_pdf()
        
#     The above example will create a PDF file named 'output.pdf' with two pages, one for Alice and one for Bob, including their respective details.
#     """
#     def __init__(self, output_pdf):
#         self.pdf_pages = PdfPages(output_pdf)
        
#     @staticmethod
#     def estimate_fig_dimensions(text, char_width=0.1, line_height=0.2):
#         lines = text.split('\n')
#         max_line_length = max(len(line) for line in lines)
#         num_lines = len(lines)
#         fig_width = max_line_length * char_width
#         fig_height = num_lines * line_height
#         return fig_width, fig_height
    
#     def add_text_to_pdf(self, doc_text):
#         fig_width, fig_height = self.estimate_fig_dimensions(doc_text)
#         fig, ax = plt.subplots(figsize=(max(8.5, fig_width), max(11, fig_height)))
#         half_inch_in_points = 0.5 * fig.dpi
#         x_position = half_inch_in_points / fig.bbox.width
#         ax.text(x_position, 1.0, doc_text, fontsize=10, ha='left', va='top', transform=ax.transAxes, family='monospace')
#         ax.axis('off')
#         self.pdf_pages.savefig(fig)
#         plt.close(fig)
    
#     @staticmethod
#     def create_document(row):
#         document = []
#         for col_name, col_value in row.items():
#             if col_name == 'code':
#                 document.append(f"\n# {col_name}:\n{col_value}\n###")
#             else:
#                 commented_value = '\n'.join(f'{line}' for line in str(col_value).split('\n'))
#                 document.append(f"\n{col_name}:\n{commented_value}\n")
#         return '\n'.join(document)
    
#     def create_pdf(self, data):
#         for index, row in data.iterrows():
#             doc_text = self.create_document(row)
#             self.add_text_to_pdf(doc_text)
    
#     def close_pdf(self):
#         self.pdf_pages.close()


In [111]:
# stretchy_pdf = StretchyPDF('./data/prepared_columns.pdf')
# stretchy_pdf.create_pdf(described_prepared_columns)
# stretchy_pdf.close_pdf()